# Credit Risk Classification

Credit risk poses a classification problem that’s inherently imbalanced. This is because healthy loans easily outnumber risky loans. In this Challenge, you’ll use various techniques to train and evaluate models with imbalanced classes. You’ll use a dataset of historical lending activity from a peer-to-peer lending services company to build a model that can identify the creditworthiness of borrowers.

## Instructions:

This challenge consists of the following subsections:

* Split the Data into Training and Testing Sets

* Create a Logistic Regression Model with the Original Data

* Predict a Logistic Regression Model with Resampled Training Data 

### Split the Data into Training and Testing Sets

Open the starter code notebook and then use it to complete the following steps.

1. Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

2. Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

    > **Note** A value of `0` in the “loan_status” column means that the loan is healthy. A value of `1` means that the loan has a high risk of defaulting.  

3. Check the balance of the labels variable (`y`) by using the `value_counts` function.

4. Split the data into training and testing datasets by using `train_test_split`.

### Create a Logistic Regression Model with the Original Data

Employ your knowledge of logistic regression to complete the following steps:

1. Fit a logistic regression model by using the training data (`X_train` and `y_train`).

2. Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

3. Evaluate the model’s performance by doing the following:

    * Calculate the accuracy score of the model.

    * Generate a confusion matrix.

    * Print the classification report.

4. Answer the following question: How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

### Predict a Logistic Regression Model with Resampled Training Data

Did you notice the small number of high-risk loan labels? Perhaps, a model that uses resampled data will perform better. You’ll thus resample the training data and then reevaluate the model. Specifically, you’ll use `RandomOverSampler`.

To do so, complete the following steps:

1. Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

2. Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

3. Evaluate the model’s performance by doing the following:

    * Calculate the accuracy score of the model.

    * Generate a confusion matrix.

    * Print the classification report.
    
4. Answer the following question: How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

### Write a Credit Risk Analysis Report

For this section, you’ll write a brief report that includes a summary and an analysis of the performance of both machine learning models that you used in this challenge. You should write this report as the `README.md` file included in your GitHub repository.

Structure your report by using the report template that `Starter_Code.zip` includes, and make sure that it contains the following:

1. An overview of the analysis: Explain the purpose of this analysis.


2. The results: Using bulleted lists, describe the balanced accuracy scores and the precision and recall scores of both machine learning models.

3. A summary: Summarize the results from the machine learning models. Compare the two versions of the dataset predictions. Include your recommendation for the model to use, if any, on the original vs. the resampled data. If you don’t recommend either model, justify your reasoning.

In [20]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from imblearn.metrics import classification_report_imbalanced

import warnings
warnings.filterwarnings('ignore')

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [13]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df_input_data = pd.read_csv(                  # Read the specified csv file into the dataframe
    Path("../../Resources/lending_data.csv")  #  using a normalized version of the path and filename for the operating system on which the code is running
)

# Review the DataFrame
display( "*** Head ***", df_input_data.head() );  # Add a heading and show the first few rows of the dataframe
display( "*** Checking for Nulls ***", df_input_data.isna().sum() );  # Add a heading and show the first few rows of the dataframe


'*** Head ***'

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


'*** Checking for Nulls ***'

loan_size           0
interest_rate       0
borrower_income     0
debt_to_income      0
num_of_accounts     0
derogatory_marks    0
total_debt          0
loan_status         0
dtype: int64

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [14]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_input_data["loan_status"]  # Create the y (labels set) from the loan_status column of the dataset

# Separate the X variable, the features
X = df_input_data.drop(columns=["loan_status"]).copy() # Create the X (features set) by copying the DF and dropping the Labels set

In [16]:
# Review the y variable Series
y[:5] # Preview 5 rows of y

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [17]:
# Review the X variable DataFrame
X.head() # Preview 5 rows of X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [19]:
# Check the balance of our target values
y.value_counts() # Show a count of each different value in the labels variable (y)

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [21]:
# Split data into into random Train and Test subsets.
X_train, X_test, y_train, y_test = train_test_split(X, y, # Split the X and y input data into random Train and Test subsets.
                                                    random_state=1) # Set random_state to ensure results are the same when repeating execution

In [26]:
display("*** X_train Info ***", X_train.info()) # Show information about the data subset

<class 'pandas.core.frame.DataFrame'>
Index: 58152 entries, 29175 to 5192
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         58152 non-null  float64
 1   interest_rate     58152 non-null  float64
 2   borrower_income   58152 non-null  int64  
 3   debt_to_income    58152 non-null  float64
 4   num_of_accounts   58152 non-null  int64  
 5   derogatory_marks  58152 non-null  int64  
 6   total_debt        58152 non-null  int64  
dtypes: float64(3), int64(4)
memory usage: 3.5 MB


'*** X_train Info ***'

None

In [27]:
display("*** X_test Info ***", X_test.info()) # Show information about the data subset

<class 'pandas.core.frame.DataFrame'>
Index: 19384 entries, 60914 to 2793
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         19384 non-null  float64
 1   interest_rate     19384 non-null  float64
 2   borrower_income   19384 non-null  int64  
 3   debt_to_income    19384 non-null  float64
 4   num_of_accounts   19384 non-null  int64  
 5   derogatory_marks  19384 non-null  int64  
 6   total_debt        19384 non-null  int64  
dtypes: float64(3), int64(4)
memory usage: 1.2 MB


'*** X_test Info ***'

None

In [29]:
display("*** y_train Info ***", y_train.info()) # Show information about the data subset

<class 'pandas.core.series.Series'>
Index: 58152 entries, 29175 to 5192
Series name: loan_status
Non-Null Count  Dtype
--------------  -----
58152 non-null  int64
dtypes: int64(1)
memory usage: 908.6 KB


'*** y_train Info ***'

None

In [30]:
display("*** y_test Info ***", y_test.info()) # Show information about the data subset

<class 'pandas.core.series.Series'>
Index: 19384 entries, 60914 to 2793
Series name: loan_status
Non-Null Count  Dtype
--------------  -----
19384 non-null  int64
dtypes: int64(1)
memory usage: 302.9 KB


'*** y_test Info ***'

None

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
# YOUR CODE HERE!

# Fit the model using training data
# YOUR CODE HERE!

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
# YOUR CODE HERE!

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
# YOUR CODE HERE!

0.9520479254722232

In [11]:
# Generate a confusion matrix for the model
# YOUR CODE HERE!

array([[18663,   102],
       [   56,   563]])

In [12]:
# Print the classification report for the model
# YOUR CODE HERE!

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.91      1.00      0.95      0.91     18765
          1       0.85      0.91      0.99      0.88      0.95      0.90       619

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** WRITE YOUR ANSWER HERE!

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [13]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
# YOUR CODE HERE!

# Fit the original training data to the random_oversampler model
# YOUR CODE HERE!

In [14]:
# Count the distinct values of the resampled labels data
# YOUR CODE HERE!

1    56271
0    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [15]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
# YOUR CODE HERE!

# Fit the model using the resampled training data
# YOUR CODE HERE!

# Make a prediction using the testing data
# YOUR CODE HERE!

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [16]:
# Print the balanced_accuracy score of the model 
# YOUR CODE HERE!

0.9936781215845847

In [17]:
# Generate a confusion matrix for the model
# YOUR CODE HERE!

array([[18649,   116],
       [    4,   615]])

In [18]:
# Print the classification report for the model
# YOUR CODE HERE!

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.99      1.00      0.99      0.99     18765
          1       0.84      0.99      0.99      0.91      0.99      0.99       619

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** YOUR ANSWER HERE!